In [38]:
import pandas as pd

In [39]:
with open('../Data/coursera_sessions_train.txt', 'r') as f:
    train_data = f.readlines()

In [40]:
with open('../Data/coursera_sessions_test.txt', 'r') as f:
    test_data = f.readlines()

In [44]:
import numpy as np
def preprocess_data(data):
    print "tdata"
    tdata = map(lambda x: x.replace('\n',''), data)
    tdata = map(lambda x: x.split(';'), tdata)
    
    print "ttdata"
    ttdata = []
    for el in tdata:
        if el[1]!='':
            ttdata.append(el)
            
    obs = [x[0].split(',') for x in ttdata]
    buy = [x[1].split(',') for x in ttdata]
    
    print "full"
    obs_full = [x[0].split(',') for x in tdata]
    buy_full = [x[1].split(',') for x in tdata]
    
    print "obs_all"
    obs_all = []
    for el in obs_full:
        obs_all += el
        
    print "buy_all"
    buy_all = []
    for el in buy_full:
        buy_all += el
    
    print "unique, counts, inds"
    obs_un, obs_inds, obs_counts = np.unique(obs_all, return_index=True, return_counts=True)
    buy_un, buy_inds, buy_counts = np.unique(buy_all, return_index=True, return_counts=True)
    
    #print "counts"
    #obs_counts = [obs_all.count(x) for x in obs_un]
    #buy_counts = [buy_all.count(x) for x in buy_un]
    
    #print "inds"
    #obs_inds = [obs_all.index(x) for x in obs_un]
    #buy_inds = [buy_all.index(x) for x in buy_un]
    
    #print "ns"
    #n_obs = [len(x) for x in obs]
    #n_buy = [len(x) for x in buy]
    return {'obs': obs,
            'obs_full': obs_full, 'obs_all': obs_all, 'obs_un': obs_un, 'obs_counts': obs_counts, #'n_obs': n_obs, 
            'obs_inds': obs_inds,
            'buy': buy,
            'buy_full': buy_full, 'buy_all': buy_all, 'buy_un': buy_un, 'buy_counts': buy_counts, #'n_buy': n_buy, 
            'buy_inds': buy_inds}

In [45]:
train_sum = preprocess_data(train_data)

tdata
ttdata
full
obs_all
buy_all
unique, counts, inds


In [46]:
pop_obs = sorted(zip(train_sum['obs_un'], train_sum['obs_counts'], train_sum['obs_inds']), key=lambda x: (-x[1], x[2]))
ppop_obs = [x[0] for x in pop_obs]
pop_buy = sorted(zip(train_sum['buy_un'], train_sum['buy_counts'], train_sum['buy_inds']), key=lambda x: (-x[1],x[2]))
ppop_buy = [x[0] for x in pop_buy]

In [69]:
def prec_recall(obs, buy, ppop_obs, k):
    #print obs
    obs_un = np.unique(obs)
    #print obs_un
    inds = map(lambda x: ppop_obs.index(x) if x in ppop_obs else 9999999, obs_un)
    indsr = map(lambda x: obs.index(x), obs_un)
    #print inds
    inds_obs = sorted(zip(inds, obs_un, indsr), key=lambda x: (x[0], x[2]))
    l = k#min(k, len(obs_un))
    recoms = [str(x[1]) for x in inds_obs][:l]
    
    #pr_sum = 0
    #rec_sum = 0
    n_sum = 0
    #for i in range(l):
    #    if recoms[i] in buy:
    #        n_sum += 1
    #        pr_sum += float(n_sum)/(i+1)
    #        rec_sum += float(n_sum)/len(buy)
    #if n_sum ==0: n_sum = 1
    #pr = float(pr_sum)/n_sum
    #recall = float(rec_sum)/n_sum
    
    for rec in recoms:
        if rec in buy:
            n_sum += 1
    pr = float(n_sum)/l
    recall = float(n_sum)/len(buy)        
    return (pr, recall)

In [52]:
prs = []
recalls = []
for i in range(len(train_sum['obs'])):
    pr, recall = prec_recall(train_sum['obs'][i], train_sum['buy'][i], ppop_obs, 1)
    prs.append(pr)
    recalls.append(recall)
print np.mean(prs)
print np.mean(recalls)

0.506097560976
0.438051907135


In [53]:
prs = []
recalls = []
for i in range(len(train_sum['obs'])):
    pr, recall = prec_recall(train_sum['obs'][i], train_sum['buy'][i], ppop_obs, 5)
    prs.append(pr)
    recalls.append(recall)
print np.mean(prs)
print np.mean(recalls)

0.211973392461
0.822482228753


In [19]:
def write_file(filename, res):
    with open(filename, 'w') as f:
        f.write(str(res))

In [61]:
write_file('recom_1.txt', ' '.join(map(str, [0.44, 0.51, 0.82, 0.21])))

In [70]:
def prec_recall_buy(obs, buy, ppop_buy, k):
    #print obs
    obs_un = np.unique(obs)
    #print obs_un
    inds = map(lambda x: ppop_buy.index(x) if x in ppop_buy else 999999, obs_un)
    indsr = map(lambda x: obs.index(x), obs_un)
    #print inds
    inds_obs = sorted(zip(inds, obs_un, indsr), key=lambda x: (x[0], x[2]))
    l = k#min(k, len(obs_un))
    recoms = [str(x[1]) for x in inds_obs][:l]
    n_sum = 0
    for rec in recoms:
        if rec in buy:
            n_sum += 1
    pr = float(n_sum)/l
    recall = float(n_sum)/len(buy)        
    return (pr, recall)

In [66]:
prs = []
recalls = []
for i in range(len(train_sum['obs'])):
    pr, recall = prec_recall_buy(train_sum['obs'][i], train_sum['buy'][i], ppop_buy, 1)
    prs.append(pr)
    recalls.append(recall)
print np.mean(prs)
print np.mean(recalls)

0.790188470067
0.67678493747


In [67]:
prs = []
recalls = []
for i in range(len(train_sum['obs'])):
    pr, recall = prec_recall_buy(train_sum['obs'][i], train_sum['buy'][i], ppop_buy, 5)
    prs.append(pr)
    recalls.append(recall)
print np.mean(prs)
print np.mean(recalls)

0.252050997783
0.925502873112


In [68]:
write_file('recom_3.txt', ' '.join(map(str, [0.68, 0.79, 0.93, 0.25])))

### test

In [ ]:
test_sum = preprocess_data(test_data)

In [72]:
prs = []
recalls = []
for i in range(len(test_sum['obs'])):
    pr, recall = prec_recall(test_sum['obs'][i], test_sum['buy'][i], ppop_obs, 1)
    prs.append(pr)
    recalls.append(recall)
print np.mean(prs)
print np.mean(recalls)

0.477762619372
0.414556075272


In [73]:
prs = []
recalls = []
for i in range(len(test_sum['obs'])):
    pr, recall = prec_recall(test_sum['obs'][i], test_sum['buy'][i], ppop_obs, 5)
    prs.append(pr)
    recalls.append(recall)
print np.mean(prs)
print np.mean(recalls)

0.203929058663
0.799589058883


In [74]:
write_file('recom_2.txt', ' '.join(map(str, [0.41, 0.48, 0.8, 0.2])))

In [75]:
prs = []
recalls = []
for i in range(len(test_sum['obs'])):
    pr, recall = prec_recall_buy(test_sum['obs'][i], test_sum['buy'][i], ppop_buy, 1)
    prs.append(pr)
    recalls.append(recall)
print np.mean(prs)
print np.mean(recalls)

0.52551159618
0.458761637898


In [76]:
prs = []
recalls = []
for i in range(len(test_sum['obs'])):
    pr, recall = prec_recall_buy(test_sum['obs'][i], test_sum['buy'][i], ppop_buy, 5)
    prs.append(pr)
    recalls.append(recall)
print np.mean(prs)
print np.mean(recalls)

0.209986357435
0.820141958533


In [77]:
write_file('recom_4.txt', ' '.join(map(str, [0.46, 0.53, 0.82, 0.21])))